# Quest to Delta Optimisation - Fabric

We will explore the following:
 - V-Order

*NB:* 
- *Demo focuses on the Speed of Creating and Flexiblity of the techniques rather than Query Performance!*

## **Setup**

Let's start by cleaning up the environment.

### Delete Exsiting Tables & Files

In [23]:
DROP TABLE IF EXISTS delta_optimize.customer;
DROP TABLE IF EXISTS delta_optimize.customer_vo;

StatementMeta(, 4502ef8f-5222-4b18-8238-f9b57389b830, 44, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

## **V-Order**

V-Order is a write time optimization to the parquet file format that enables fast reads.

### **Delta table <u>without</u> V-Order**

Create and query a Delta table with V-Order disabled.

#### **Create and Populate a Delta table**

- Disable `V-Order`
- Create Delta table
- Insert **`Dummy`** Data into Delta table

##### Disable `V-Order`

In [24]:
SET spark.sql.parquet.vorder.enabled=FALSE

StatementMeta(, 4502ef8f-5222-4b18-8238-f9b57389b830, 45, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 2 fields>

##### Create Delta table

In [25]:
-- Create Table
CREATE TABLE IF NOT EXISTS delta_optimize.customer
  (ID INT, customer_name STRING, street STRING, city STRING, district STRING, state STRING, postcode STRING, region STRING)
USING DELTA;

StatementMeta(, 4502ef8f-5222-4b18-8238-f9b57389b830, 46, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

##### Insert Data into Delta table


**RESULT:** *Runtime ~52 sec*

In [26]:
-- Insert Data
INSERT INTO delta_optimize.customer(ID, customer_name, street, city, district, state, postcode, region)
	VALUES (1, 'Alice Johnson', '123 Maple St', 'London', 'Camden', 'Greater London', 'NW1 5DB', 'England');
INSERT INTO delta_optimize.customer(ID, customer_name, street, city, district, state, postcode, region)
	VALUES (2, 'Bob Smith', '456 Oak Ave', 'Manchester', 'Didsbury', 'Greater Manchester', 'M20 2UF', 'England');
INSERT INTO delta_optimize.customer(ID, customer_name, street, city, district, state, postcode, region)
	VALUES (3, 'Charlie Brown', '789 Pine Rd', 'Birmingham', 'Edgbaston', 'West Midlands', 'B15 2TT', 'England');
INSERT INTO delta_optimize.customer(ID, customer_name, street, city, district, state, postcode, region)
	VALUES (4, 'David Miller', '101 Birch Blvd', 'Edinburgh', 'Leith', 'City of Edinburgh', 'EH6 6NZ', 'Scotland');
INSERT INTO delta_optimize.customer(ID, customer_name, street, city, district, state, postcode, region)
	VALUES (5, 'Eva White', '202 Cedar Ln', 'Glasgow', 'Hillhead', 'Glasgow City', 'G12 8QQ', 'Scotland');
INSERT INTO delta_optimize.customer(ID, customer_name, street, city, district, state, postcode, region)
	VALUES (6, 'Frank Harris', '303 Elm St', 'Bristol', 'Clifton', 'Bristol', 'BS8 1UD', 'England');
INSERT INTO delta_optimize.customer(ID, customer_name, street, city, district, state, postcode, region)
	VALUES (7, 'Grace Lee', '404 Willow Dr', 'Liverpool', 'Wavertree', 'Merseyside', 'L15 1HN', 'England');
INSERT INTO delta_optimize.customer(ID, customer_name, street, city, district, state, postcode, region)
	VALUES (8, 'Henry Clark', '505 Aspen Ct', 'Cardiff', 'Roath', 'Cardiff', 'CF24 3EX', 'Wales');
INSERT INTO delta_optimize.customer(ID, customer_name, street, city, district, state, postcode, region)
	VALUES (9, 'Isabella Davis', '606 Spruce Ave', 'Belfast', 'Botanic', 'Belfast', 'BT7 1NN', 'Northern Ireland');
INSERT INTO delta_optimize.customer(ID, customer_name, street, city, district, state, postcode, region)
	VALUES (10, 'Jack Wilson', '707 Redwood Way', 'Oxford', 'Headington', 'Oxfordshire', 'OX3 9DU', 'England');

StatementMeta(, 4502ef8f-5222-4b18-8238-f9b57389b830, 56, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

...

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

#### **Query the Delta table**

Query and observe the new execution run time

**RESULT:** *Runtime ~6 seconds*

In [27]:
SELECT *
FROM delta_optimize.customer

StatementMeta(, 4502ef8f-5222-4b18-8238-f9b57389b830, 57, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 8 fields>

#### **Query the Delta table metadata**

Get the table Metadata using **`DESCRIBE DETAIL`** statement

**RESULT:** *NumFiles = `10`*

In [28]:
DESCRIBE DETAIL delta_optimize.customer

StatementMeta(, 4502ef8f-5222-4b18-8238-f9b57389b830, 58, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 14 fields>

#### **Query the Delta files**

In [29]:
%%pyspark

mssparkutils.fs.ls("Tables/customer")

StatementMeta(, 4502ef8f-5222-4b18-8238-f9b57389b830, 59, Finished, Available, Finished)

[FileInfo(path=abfss://50c55be7-4da3-490d-9c86-9314a3e32f8c@onelake.dfs.fabric.microsoft.com/0c12c4f6-13d0-4783-8479-d48f32286840/Tables/customer/_delta_log, name=_delta_log, size=0),
 FileInfo(path=abfss://50c55be7-4da3-490d-9c86-9314a3e32f8c@onelake.dfs.fabric.microsoft.com/0c12c4f6-13d0-4783-8479-d48f32286840/Tables/customer/part-00000-03f573b5-ad22-4783-b5c0-8df58f3301ff-c000.snappy.parquet, name=part-00000-03f573b5-ad22-4783-b5c0-8df58f3301ff-c000.snappy.parquet, size=3947),
 FileInfo(path=abfss://50c55be7-4da3-490d-9c86-9314a3e32f8c@onelake.dfs.fabric.microsoft.com/0c12c4f6-13d0-4783-8479-d48f32286840/Tables/customer/part-00000-44b1ebed-2bf6-4dda-b3fa-c18f0e328176-c000.snappy.parquet, name=part-00000-44b1ebed-2bf6-4dda-b3fa-c18f0e328176-c000.snappy.parquet, size=3898),
 FileInfo(path=abfss://50c55be7-4da3-490d-9c86-9314a3e32f8c@onelake.dfs.fabric.microsoft.com/0c12c4f6-13d0-4783-8479-d48f32286840/Tables/customer/part-00000-602aca97-45e3-46c0-b8e4-2b5abc21458a-c000.snappy.parquet,

### **Delta Table <u>with</u> V-Order**


Create and query a Delta table with V-Order Enabled.

#### **Create and Populate a Delta table**

- Enable `V-Order`
- Create Delta table
- Insert Data into Delta Table

##### Enable `V-Order`

In [30]:
SET spark.sql.parquet.vorder.enabled = TRUE

StatementMeta(, 4502ef8f-5222-4b18-8238-f9b57389b830, 60, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 2 fields>

##### Create Delta table

In [31]:
-- Create Table
CREATE TABLE IF NOT EXISTS delta_optimize.customer_vo
  (ID INT, customer_name STRING, street STRING, city STRING, district STRING, state STRING, postcode STRING, region STRING)
USING DELTA;

StatementMeta(, 4502ef8f-5222-4b18-8238-f9b57389b830, 61, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

##### Insert Data into Delta table


**RESULT:** *Runtime ~30 sec*

In [32]:
INSERT INTO delta_optimize.customer_vo(ID, customer_name, street, city, district, state, postcode, region)
	VALUES (1, 'Alice Johnson', '123 Maple St', 'London', 'Camden', 'Greater London', 'NW1 5DB', 'England');
INSERT INTO delta_optimize.customer_vo(ID, customer_name, street, city, district, state, postcode, region)
	VALUES (2, 'Bob Smith', '456 Oak Ave', 'Manchester', 'Didsbury', 'Greater Manchester', 'M20 2UF', 'England');
INSERT INTO delta_optimize.customer_vo(ID, customer_name, street, city, district, state, postcode, region)
	VALUES (3, 'Charlie Brown', '789 Pine Rd', 'Birmingham', 'Edgbaston', 'West Midlands', 'B15 2TT', 'England');
INSERT INTO delta_optimize.customer_vo(ID, customer_name, street, city, district, state, postcode, region)
	VALUES (4, 'David Miller', '101 Birch Blvd', 'Edinburgh', 'Leith', 'City of Edinburgh', 'EH6 6NZ', 'Scotland');
INSERT INTO delta_optimize.customer_vo(ID, customer_name, street, city, district, state, postcode, region)
	VALUES (5, 'Eva White', '202 Cedar Ln', 'Glasgow', 'Hillhead', 'Glasgow City', 'G12 8QQ', 'Scotland');
INSERT INTO delta_optimize.customer_vo(ID, customer_name, street, city, district, state, postcode, region)
	VALUES (6, 'Frank Harris', '303 Elm St', 'Bristol', 'Clifton', 'Bristol', 'BS8 1UD', 'England');
INSERT INTO delta_optimize.customer_vo(ID, customer_name, street, city, district, state, postcode, region)
	VALUES (7, 'Grace Lee', '404 Willow Dr', 'Liverpool', 'Wavertree', 'Merseyside', 'L15 1HN', 'England');
INSERT INTO delta_optimize.customer_vo(ID, customer_name, street, city, district, state, postcode, region)
	VALUES (8, 'Henry Clark', '505 Aspen Ct', 'Cardiff', 'Roath', 'Cardiff', 'CF24 3EX', 'Wales');
INSERT INTO delta_optimize.customer_vo(ID, customer_name, street, city, district, state, postcode, region)
	VALUES (9, 'Isabella Davis', '606 Spruce Ave', 'Belfast', 'Botanic', 'Belfast', 'BT7 1NN', 'Northern Ireland');
INSERT INTO delta_optimize.customer_vo(ID, customer_name, street, city, district, state, postcode, region)
	VALUES (10, 'Jack Wilson', '707 Redwood Way', 'Oxford', 'Headington', 'Oxfordshire', 'OX3 9DU', 'England');

StatementMeta(, 4502ef8f-5222-4b18-8238-f9b57389b830, 71, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

...

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

#### **Query the Delta table**

Query and observe the new execution run time

**RESULT:** *Runtime ~2 seconds*

In [33]:
SELECT *
FROM delta_optimize.customer_vo


StatementMeta(, 4502ef8f-5222-4b18-8238-f9b57389b830, 72, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 8 fields>

#### **Query the Delta table metadata**

Get the table Metadata using **`DESCRIBE DETAIL`** statement

**RESULT:** *NumFiles = `10`*

In [34]:
DESCRIBE DETAIL delta_optimize.customer_vo

StatementMeta(, 4502ef8f-5222-4b18-8238-f9b57389b830, 73, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 14 fields>

#### **Query the Delta files**

In [35]:
%%pyspark

mssparkutils.fs.ls("Tables/customer_vo")

StatementMeta(, 4502ef8f-5222-4b18-8238-f9b57389b830, 74, Finished, Available, Finished)

[FileInfo(path=abfss://50c55be7-4da3-490d-9c86-9314a3e32f8c@onelake.dfs.fabric.microsoft.com/0c12c4f6-13d0-4783-8479-d48f32286840/Tables/customer_vo/_delta_log, name=_delta_log, size=0),
 FileInfo(path=abfss://50c55be7-4da3-490d-9c86-9314a3e32f8c@onelake.dfs.fabric.microsoft.com/0c12c4f6-13d0-4783-8479-d48f32286840/Tables/customer_vo/part-00000-0bacfd76-53c4-48e7-a7c0-01c2eb68d9d1-c000.snappy.parquet, name=part-00000-0bacfd76-53c4-48e7-a7c0-01c2eb68d9d1-c000.snappy.parquet, size=3927),
 FileInfo(path=abfss://50c55be7-4da3-490d-9c86-9314a3e32f8c@onelake.dfs.fabric.microsoft.com/0c12c4f6-13d0-4783-8479-d48f32286840/Tables/customer_vo/part-00000-1bc8e128-c290-4f7b-9367-1a3827456b90-c000.snappy.parquet, name=part-00000-1bc8e128-c290-4f7b-9367-1a3827456b90-c000.snappy.parquet, size=4032),
 FileInfo(path=abfss://50c55be7-4da3-490d-9c86-9314a3e32f8c@onelake.dfs.fabric.microsoft.com/0c12c4f6-13d0-4783-8479-d48f32286840/Tables/customer_vo/part-00000-80968b50-a943-4653-8260-19cdbccab3d0-c000.sna

## **Maintenance**

### **OPTIMIZE**

Optimize the Lakehouse with the OPTIMIZE command. 

#### **'OPTIMIZE' the Delta table**

In [36]:
OPTIMIZE delta_optimize.customer_vo VORDER;

StatementMeta(, 4502ef8f-5222-4b18-8238-f9b57389b830, 75, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 2 fields>

#### **Query the Delta files**


In [37]:
%%pyspark

mssparkutils.fs.ls("Tables/customer_vo/_delta_log")

StatementMeta(, 4502ef8f-5222-4b18-8238-f9b57389b830, 76, Finished, Available, Finished)

[FileInfo(path=abfss://50c55be7-4da3-490d-9c86-9314a3e32f8c@onelake.dfs.fabric.microsoft.com/0c12c4f6-13d0-4783-8479-d48f32286840/Tables/customer_vo/_delta_log/00000000000000000000.json, name=00000000000000000000.json, size=1255),
 FileInfo(path=abfss://50c55be7-4da3-490d-9c86-9314a3e32f8c@onelake.dfs.fabric.microsoft.com/0c12c4f6-13d0-4783-8479-d48f32286840/Tables/customer_vo/_delta_log/00000000000000000001.json, name=00000000000000000001.json, size=1181),
 FileInfo(path=abfss://50c55be7-4da3-490d-9c86-9314a3e32f8c@onelake.dfs.fabric.microsoft.com/0c12c4f6-13d0-4783-8479-d48f32286840/Tables/customer_vo/_delta_log/00000000000000000002.json, name=00000000000000000002.json, size=1191),
 FileInfo(path=abfss://50c55be7-4da3-490d-9c86-9314a3e32f8c@onelake.dfs.fabric.microsoft.com/0c12c4f6-13d0-4783-8479-d48f32286840/Tables/customer_vo/_delta_log/00000000000000000003.json, name=00000000000000000003.json, size=1191),
 FileInfo(path=abfss://50c55be7-4da3-490d-9c86-9314a3e32f8c@onelake.dfs.fabr

#### **Query the Delta JSON log**

In [39]:
%%pyspark

file_path = "Tables/customer_vo/_delta_log/00000000000000000011.json"

df = spark.read.json(file_path)
display(df)

StatementMeta(, 4502ef8f-5222-4b18-8238-f9b57389b830, 78, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1f08ce78-cdea-41a7-83c6-8a77b47508df)

#### View the Delta Files & Delta JSON Log in the Lakehouse Explorer

### **Vacuum**

Remove Stale Parquet Files with the VACUUM Command

In [40]:
SET spark.databricks.delta.retentionDurationCheck.enabled = false;

StatementMeta(, 4502ef8f-5222-4b18-8238-f9b57389b830, 79, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 2 fields>

#### **Vacuum to Remove Old Files**

Clean up stale files in Delta Lake tables using the **`VACUUM`** operation.

In [41]:
VACUUM delta_optimize.customer_vo RETAIN 0 HOURS;

StatementMeta(, 4502ef8f-5222-4b18-8238-f9b57389b830, 80, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>

#### **Query the Delta files**

**RESULT:** *The stale files removed*

In [42]:
%%pyspark

mssparkutils.fs.ls("Tables/customer_vo/")

StatementMeta(, 4502ef8f-5222-4b18-8238-f9b57389b830, 81, Finished, Available, Finished)

[FileInfo(path=abfss://50c55be7-4da3-490d-9c86-9314a3e32f8c@onelake.dfs.fabric.microsoft.com/0c12c4f6-13d0-4783-8479-d48f32286840/Tables/customer_vo/_delta_log, name=_delta_log, size=0),
 FileInfo(path=abfss://50c55be7-4da3-490d-9c86-9314a3e32f8c@onelake.dfs.fabric.microsoft.com/0c12c4f6-13d0-4783-8479-d48f32286840/Tables/customer_vo/part-00000-d1766a08-e4b4-43dd-8f40-f1165576e5e8-c000.snappy.parquet, name=part-00000-d1766a08-e4b4-43dd-8f40-f1165576e5e8-c000.snappy.parquet, size=4830)]

Notebook finished.